In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn.functional as F

# Define data transformations with enhanced augmentations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Path to your external dataset
dataset_dir = 'D:\\Programs\\Jupyter\\Pest_data'

# Load train and test datasets
train_dir = os.path.join(dataset_dir, 'train')
test_dir = os.path.join(dataset_dir, 'test')

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Update the number of classes dynamically based on the dataset
num_classes = len(train_dataset.classes)

# Define the model with improvements
class SelfAttention(nn.Module):
    def __init__(self, in_channels):
        super(SelfAttention, self).__init__()
        self.query = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.key = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.value = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        batch_size, C, H, W = x.size()
        query = self.query(x).view(batch_size, -1, H * W).permute(0, 2, 1)
        key = self.key(x).view(batch_size, -1, H * W)
        attention = torch.bmm(query, key)
        attention = F.softmax(attention, dim=-1)

        value = self.value(x).view(batch_size, -1, H * W)
        out = torch.bmm(value, attention.permute(0, 2, 1))
        out = out.view(batch_size, C, H, W)

        out = self.gamma * out + x
        return out

class PestIdentificationModel(nn.Module):
    def __init__(self, num_classes):
        super(PestIdentificationModel, self).__init__()
        resnet = models.resnet50(pretrained=True)
        
        # Freeze initial layers for better fine-tuning
        for param in resnet.parameters():
            param.requires_grad = False
        
        self.resnet_features = nn.Sequential(*list(resnet.children())[:-2])
        self.self_attention = SelfAttention(in_channels=2048)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.5)  # Dropout to reduce overfitting
        self.fc = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.resnet_features(x)
        x = self.self_attention(x)
        x = self.avg_pool(x).view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

model = PestIdentificationModel(num_classes=num_classes)
model = model.cuda()  # Move model to GPU

# Define optimizer, loss function, and learning rate scheduler
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

# Training loop with improvements
epochs = 15
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss}")
    scheduler.step(avg_loss)  # Adjust learning rate based on loss

# Test the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.cuda(), labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy}%")

Epoch 1/15, Loss: 0.9774919096110524
Epoch 2/15, Loss: 0.5311626286610313
Epoch 3/15, Loss: 0.4716291250526041
Epoch 4/15, Loss: 0.4495019508336765
Epoch 5/15, Loss: 0.43360744335729146
Epoch 6/15, Loss: 0.3987090182153211
Epoch 7/15, Loss: 0.40189534101797186
Epoch 8/15, Loss: 0.39402336087347806
Epoch 9/15, Loss: 0.3866499874038973
Epoch 10/15, Loss: 0.37965307449517044
Epoch 11/15, Loss: 0.37068800412226416
Epoch 12/15, Loss: 0.3679426585634549
Epoch 13/15, Loss: 0.37225163576827536
Epoch 14/15, Loss: 0.3724978964343883
Epoch 15/15, Loss: 0.36834953833317413
Test Accuracy: 83.62147406733393%
